In [1]:
from media_impact_monitor.data_loaders.donations.opencollective import (
    fetch_climate_orgs,
    fetch_transactions,
    get_transactions_df,
    get_monthly_dfs,
    group_by_wealth,
    generalize_group_name
)

orgs = fetch_climate_orgs()
orgs[0]


Found 901 unique organizations


{'slug': 'climate-caucus',
 'name': 'Climate Caucus',
 'legalName': None,
 'description': 'Connect elected Canadian representatives to drive climate action',
 'longDescription': None,
 'tags': ['community'],
 'location': None,
 'stats': {'totalAmountReceived': {'value': 0,
   'currency': 'USD',
   'valueInCents': 0},
  'totalAmountReceivedTimeSeries': {'dateFrom': None,
   'dateTo': None,
   'timeUnit': 'YEAR',
   'nodes': []}}}

In [2]:
transactions = fetch_transactions(orgs[1]["slug"])
transactions[:4]

[{'id': 'b04b2277-1fb2-4a91-a682-26296a21cf9d',
  'createdAt': '2024-07-19T11:43:33.884Z',
  'type': 'DEBIT',
  'amount': {'value': -28.78, 'currency': 'GBP'}},
 {'id': '463bbc47-e299-418f-b021-7d9a8cd99717',
  'createdAt': '2023-12-15T08:21:06.535Z',
  'type': 'DEBIT',
  'amount': {'value': -26.38, 'currency': 'GBP'}},
 {'id': 'c1f4c332-d51a-49af-8cdf-e2e7ca9902e7',
  'createdAt': '2023-03-29T12:03:06.051Z',
  'type': 'DEBIT',
  'amount': {'value': -42, 'currency': 'GBP'}},
 {'id': '0c936cd0-05fb-49a4-9692-8dec4cdaa501',
  'createdAt': '2023-03-29T12:03:04.254Z',
  'type': 'DEBIT',
  'amount': {'value': -22, 'currency': 'GBP'}}]

In [3]:
df = get_transactions_df(orgs[:10])
df

  0%|          | 0/10 [00:00<?, ?it/s]

,date,amount,organization
0,2024-07-19 00:00:00+00:00,-28.78,Climate Hub Wandsworth
1,2023-12-15 00:00:00+00:00,-26.38,Climate Hub Wandsworth
2,2023-03-29 00:00:00+00:00,-42.00,Climate Hub Wandsworth
3,2023-03-29 00:00:00+00:00,-22.00,Climate Hub Wandsworth
4,2022-09-21 00:00:00+00:00,-0.90,Climate Hub Wandsworth
...,...,...,...
30,2024-02-12 00:00:00+00:00,5.00,Climate Collective Oxford
31,2024-01-19 00:00:00+00:00,-42.00,Climate Collective Oxford
32,2024-01-18 00:00:00+00:00,-2.50,Climate Collective Oxford
33,2024-01-18 00:00:00+00:00,-1.03,Climate Collective Oxford


In [4]:
df = get_transactions_df(orgs[:150])
df["organization"] = df["organization"].apply(generalize_group_name)
df = group_by_wealth(df, top_n=8)
positive, negative = get_monthly_dfs(df)

  0%|          | 0/150 [00:00<?, ?it/s]

In [5]:
positive

,organization,date,amount
0,Climate Crisis Collective,2019-08-25 00:00:00+00:00,25000.00
2,Climate Crisis Collective,2019-09-15 00:00:00+00:00,200.00
5,Climate Crisis Collective,2019-10-06 00:00:00+00:00,25000.00
8,Climate Crisis Collective,2019-12-01 00:00:00+00:00,13500.00
10,Climate Crisis Collective,2019-12-22 00:00:00+00:00,466.00
...,...,...,...
1017,The Xylom,2024-09-15 00:00:00+00:00,17696.96
1020,The Xylom,2024-10-13 00:00:00+00:00,13.26
1022,The Xylom,2024-10-27 00:00:00+00:00,6.67
1026,The Xylom,2024-12-01 00:00:00+00:00,19.95


In [15]:
import altair as alt


alt.Chart(positive).mark_area().encode(
    x="date:T",  # T: temporal
    y="amount:Q",  # Q: quantitative
    color="organization:N",  # N: nominal
    tooltip=["date", "amount"],
).properties(width=600, height=100).facet(
    row='organization:N'
).show()

alt.FacetChart(...)